In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

blob_doh = skimage.feature.blob_doh
blob_dog = skimage.feature.blob_dog

In [3]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                           classes, transitions, times): 
    
   
    
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    phighlight = binary_opening(thresh_img, square(14))
    img = np.copy(frame)
    img[phighlight==0] = 0
    pblobs =blob_doh(img, min_sigma = 10, max_sigma = 16, threshold = 0.007, num_sigma= 15, overlap=0.6)
        
    ghighlight = opening(
        white_tophat(gaussian(thresh_img, sigma=0.2), square(14)),
                                   disk(4))
    img = np.copy(frame)
    img[ghighlight==0] = 0
    gblobs =blob_dog(img, min_sigma = 8, max_sigma = 12, threshold = 0.2, overlap=0.5)
    

    ngblobs = len(gblobs)
    npblobs = len(pblobs) 
    num_blobs = ngblobs + npblobs
            
    plas_on_glas_dists =[connect((pblob[1], pblob[0]),m_bond, gblobs) for pblob in pblobs]
    num_mconnections = reducer(plas_on_glas_dists)


    glas_on_glas_dists =[connect((gblobs[i][1], gblobs[i][0]),g_bond, gblobs[i+1:]) for i in range(ngblobs-1)]
    num_gconnections = reducer(glas_on_glas_dists)



    num_connections = sum([num_gconnections,num_mconnections])
    
    #----------------------filter------------------------- can use actual filter
    test = (num_connections >= expected_connections and num_blobs>=expected_blobs)
    if test: # keep frames that have enough paricles, and are not transitions
        clas =sideify(num_gconnections, num_mconnections )
        classes[clas]+=1 
            #print(clas)
        
        if clas == 'ucf':
            #broken_count += 1
            #class_num = 0
            #print('ucf')
            pass
        else:
            if origin == 'ucf':
                origin = last_class
                #print("reset origin", origin)
            if clas == last_class:
                class_num += 1
                #print("class_num: %s" %class_num)
            else:
                class_num = 0
                #print("class_reset")
                #print("class_num: %s" %class_num)
            if class_num >= class_thresh: 
                if broken_count >= transition_threshold or clas != origin:
                    transitions[origin + "->" + clas]+=1
                    
                    between_time = frame_no - state_start                               
                    times[origin + "->" + clas][0].append(between_time)  

                    stable_time = last_whole - state_start
                    times[origin + "->" + clas][1].append(stable_time)

                    unstable_time = frame_no - last_whole                      
                    times[origin + "->" + clas][2].append(unstable_time)
                
                    state_start = frame_no
                    
                    #print(origin + "->" + clas)
                    origin = clas
                    #print("origin: %s" %origin)
                broken_count = 0
                #print("broken_count: %s" %broken_count)    
                last_whole = frame_no
                
            filtrate_len += 1 
            last_class = clas
            
            
    else:
        #print("broken")
        class_num = 0
        #print("class_num: %s" %class_num)
        broken_count += 1
        #print("broken_count: %s" %broken_count)

    return [broken_count, class_num, origin, last_class, last_whole,state_start, filtrate_len]



        
# take largest isodata image region, and return aclassification
def sideify( num_gconnections, num_mconnections):    
    if num_mconnections == 2 and num_gconnections ==3:# and num_mconnections==4: #gtotallengths>48
        return "t"
    elif num_mconnections == 3 and num_gconnections ==2:# and num_mconnections == 4:
        return "c"
    else:
        return "ucf"

def reducer(connections):
    return sum(connections)    
def connect(cv, radius, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    norm_radius = 15
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        if vector_d <= 2*radius and vector_d > .2*radius: # blob centers closer than diameter of one blob
            num_connections += 1
        else:
            pass
    return (num_connections)

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

In [4]:

"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/De*styrene/*[6].avi")# +glob("./tchakamau/W*half/*_[1][0,1,2,3,4,7,8,9]*.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)
num_top_keys = 3
num_full_keys = 3
num_tran_keys = 2
num_lengths = 3
trans_threshes = np.linspace(50, 100, 3)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes = np.linspace(5, 60, 10)#[15, 22, 30]
trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
g_bond = 15*1
m_bond = 15*1.15
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, g_bond, m_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        T_transitions = [[[[[] for i in range(num_tran_keys)] for j in range(num_tran_keys)] for k in range(num_lengths)] 
                          for l in range(len(filenames))]
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            classes = [("t",0), ("c",0), ("ucf",0)]
            transitions = [("t->t",0), ("t->c",0),
                          ("c->t" ,0), ("c->c",0)]
            times = [("t->t",[[] for i in range(num_lengths)]), ("t->c",[[] for i in range(num_lengths)]),
                          ("c->t" ,[[] for i in range(num_lengths)]), ("c->c",[[] for i in range(num_lengths)])]

            transitions = OrderedDict(transitions)
            times = OrderedDict(times)            
            classes = OrderedDict(classes)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            state_start = 0
            frame_no = 0
            for fr in frames:
                frame = fr[:, :, 2]
                frame_no += 1
                num_total_frames += 1
                org = total_threshold_filter(frame, frame_no, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                                              classes, transitions, times)
                broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
            ckeys = list(classes.keys())
            clcs = [classes[x] for x in ckeys] # list of frame nums per class
            
            tkeys = list(times.keys())
            tims = [times[x] for x in tkeys]
            
            trkeys = list(transitions.keys())
            trans = [transitions[x] for x in trkeys] #num of each transition
            #ucf = len(sides["ucf"])

            #Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = clcs
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))             
            T_transitions[vidnum] = np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0])
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            print(T_transitions[vidnum])
            print(Full_classif[vidnum,:])
        trans_runs[index] = [[Full_classif, (N_transitions, T_transitions), (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys,  ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_destyrene_error.npy',trans_runs)


['./tchakamau\\Density_matchedstyrene\\45696_15Vpp_6.avi']
Number files:  1
Processing vid 0 : ./tchakamau\Density_matchedstyrene\45696_15Vpp_6.avi
T threshold: 50.0
C threshold: 5.0
[[ 6. 10.]
 [10. 23.]]
[[[list([2281, 115, 968, 677, 2723, 535])
   list([4130, 315, 3893, 1562, 3354, 1078, 2181, 56, 185, 480])]
  [list([448, 7690, 1724, 415, 1407, 402, 14133, 1154, 66, 199])
   list([273, 604, 1835, 319, 360, 371, 1064, 586, 246, 287, 2120, 636, 3939, 398, 1232, 234, 106, 461, 132, 1011, 283, 513, 1672])]]

 [[list([1954, 28, 904, 64, 2518, 310])
   list([3596, 255, 3853, 1504, 3143, 965, 2138, 19, 144, 63])]
  [list([121, 7641, 1352, 1, 900, 328, 12795, 883, 19, 6])
   list([135, 543, 1718, 2, 150, 308, 266, 517, 39, 0, 252, 541, 3366, 1, 1134, 85, 24, 389, 6, 456, 222, 143, 1216])]]

 [[list([327, 87, 64, 613, 205, 225])
   list([534, 60, 40, 58, 211, 113, 43, 37, 41, 417])]
  [list([327, 49, 372, 414, 507, 74, 1338, 271, 47, 193])
   list([138, 61, 117, 317, 210, 63, 798, 69, 207, 

T threshold: 50.0
C threshold: 60.0
[[ 4.  7.]
 [ 7. 18.]]
[[[list([2167, 2718, 3505, 679])
   list([4334, 344, 2095, 3832, 872, 2216, 1487])]
  [list([803, 2603, 642, 1243, 1195, 13917, 1101])
   list([593, 339, 1196, 655, 7382, 1564, 390, 341, 1270, 2202, 631, 614, 3412, 1180, 941, 1061, 738, 732])]]

 [[list([1544, 2035, 3299, 194])
   list([3155, 35, 1449, 2949, 40, 2008, 135])]
  [list([41, 1130, 33, 453, 2, 12434, 655])
   list([81, 104, 961, 22, 6794, 25, 45, 223, 58, 28, 404, 22, 2548, 965, 7, 49, 581, 31])]]

 [[list([623, 683, 206, 485])
   list([1179, 309, 646, 883, 832, 208, 1352])]
  [list([762, 1473, 609, 790, 1193, 1483, 446])
   list([512, 235, 235, 633, 588, 1539, 345, 118, 1212, 2174, 227, 592, 864, 215, 934, 1012, 157, 701])]]]
[17708. 47861.  3891.]
Processing vid 0 : ./tchakamau\Density_matchedstyrene\45696_15Vpp_6.avi
T threshold: 75.0
C threshold: 5.0
[[ 4. 10.]
 [10. 17.]]
[[[list([2281, 677, 2723, 535])
   list([4130, 1398, 3893, 1562, 3354, 1078, 2181, 56, 185

T threshold: 75.0
C threshold: 60.0
[[ 2.  7.]
 [ 7. 14.]]
[[[list([2167, 2718]) list([4334, 3849, 2095, 3832, 1551, 2216, 1487])]
  [list([803, 2944, 642, 1243, 1195, 13917, 1101])
   list([593, 339, 1851, 7382, 1564, 390, 1270, 2202, 631, 4026, 1180, 941, 1061, 1470])]]

 [[list([1544, 2035]) list([3155, 3540, 1449, 2949, 719, 2008, 135])]
  [list([41, 1471, 33, 453, 2, 12434, 655])
   list([81, 104, 1218, 6794, 25, 45, 58, 28, 404, 3162, 965, 7, 49, 769])]]

 [[list([623, 683]) list([1179, 309, 646, 883, 832, 208, 1352])]
  [list([762, 1473, 609, 790, 1193, 1483, 446])
   list([512, 235, 633, 588, 1539, 345, 1212, 2174, 227, 864, 215, 934, 1012, 701])]]]
[17708. 47861.  3891.]
Processing vid 0 : ./tchakamau\Density_matchedstyrene\45696_15Vpp_6.avi
T threshold: 100.0
C threshold: 5.0
[[ 4. 10.]
 [10. 15.]]
[[[list([2281, 677, 2723, 535])
   list([4130, 1398, 3893, 1562, 3354, 1078, 2181, 56, 185, 480])]
  [list([1052, 7690, 2095, 1001, 1407, 969, 14133, 1154, 66, 199])
   list([273, 